In [5]:
from data_prepare import *

In [68]:
from models import Plain_RNN, Criterion

In [45]:
import importlib

In [34]:
# Test data_prepare
train_en_w, train_cn_w = load_data(train_file)
dev_en_w, dev_cn_w = load_data(dev_file)

en2idx, idx2en = build_vocab(train_en_w)
cn2idx, idx2cn = build_vocab(train_cn_w)

train_en, train_cn = sentences2idx(train_en_w, train_cn_w, en2idx, cn2idx, len_sort=False)
dev_en, dev_cn = sentences2idx(dev_en_w, dev_cn_w, en2idx, cn2idx, len_sort=False)

In [35]:
batch_size = 64
train_data = gen_examples(train_en, train_cn, batch_size)
dev_data = gen_examples(dev_en, dev_cn, batch_size)
print('num of batchs:', len(train_data))
print('maxlen of No.200 batch:', train_data[200][0][0].size)

num of batchs: 228
maxlen of No.200 batch: 20


In [ ]:
# choose No.200 batch
batch_x = torch.from_numpy(train_data[200][0]).long()
batch_x_len = torch.from_numpy(train_data[200][1]).long()
print('idx sentences:\n', batch_x, '\n', batch_x.shape)  # 64, 20
print('len of sentences:\n', batch_x_len, '\n', batch_x_len.shape)  # 64

In [62]:
# Test Encoder
importlib.reload(Plain_RNN)
M = RNN_base.PlainEncoder(len(en2idx), 100)

output, hidden = M(batch_x, batch_x_len)
print(output.shape, hidden.shape)

torch.Size([64, 20, 100]) torch.Size([1, 64, 100])
torch.Size([64, 20, 100]) torch.Size([1, 64, 100])


In [69]:
dropout = 0.2
hidden_size=100
encoder = Plain_RNN.PlainEncoder(len(en2idx), hidden_size, dropout)
decoder = Plain_RNN.PlainDecoder(len(en2idx), hidden_size, dropout)

model = Plain_RNN.PlainSeq2Seq(encoder, decoder)
# model = model.to('cuda')

loss_fn = Criterion.LanguageModelCriterion()
optimizer = torch.optim.Adam(model.parameters())

In [71]:
def train(model, data, num_epochs):
    model.train() # 训练模式
    for epoch in range(num_epochs):
        total_num_words = total_loss = 0.
        for it, (mb_x, mb_x_len, mb_y, mb_y_len) in enumerate(data):
            mb_x = torch.from_numpy(mb_x).to(device).long()
            mb_x_len = torch.from_numpy(mb_x_len).to(device).long()
            mb_input = torch.from_numpy(mb_y[:, :-1]).to(device).long() # EOS之前
            mb_output = torch.from_numpy(mb_y[:, 1:]).to(device).long() # BOS之后
            mb_y_len = torch.from_numpy(mb_y_len - 1).to(device).long()
            mb_y_len[mb_y_len <= 0] = 1
            mb_pred, attn = model(mb_x, mb_x_len, mb_input, mb_y_len)
            # [mb_y_len.max()]->[1, mb_y_len.max()]
            mb_out_mask = torch.arange(mb_y_len.max().item(), device=device)[None, :] < mb_y_len[:, None]
            mb_out_mask = mb_out_mask.float()
            # (pre, target, mask)
            # mb_output是句子单词的索引
            loss = loss_fn(mb_pred, mb_output, mb_out_mask)
            num_words = torch.sum(mb_y_len).item()
            total_loss += loss.item() * num_words
            total_num_words += num_words
            # 更新模型
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 5.)
            optimizer.step()
            if it % 10 == 0:
                print("Epoch: ", epoch, 'iteration', it, 'loss:', loss.item())
            print("Epoch", epoch, "Training loss", total_loss / total_num_words)
            # if epoch % 5 == 0:
            #     evaluate(model, dev_data)
            #     torch.save(model.state_dict(), 'translate_model.pt')

In [74]:
device = 'cpu'

device = 'cpu'
train(model, train_data, num_epochs=20)

Epoch:  0 iteration 0 loss: 5.208959102630615
Epoch 0 Training loss 5.208959102630615
Epoch:  0 iteration 100 loss: 4.840067386627197
Epoch 0 Training loss 5.020915439092309


KeyboardInterrupt: 